| Model                            | train_loss | val_loss | test_loss | version |
|----------------------------------|------------|----------|-----------|---------|
| Initial                          |     2.0914 |   6.4632 |    1.2512 |      V1 |
| Initial (normalized)             |     0.1155 |   0.4484 |    4.7467 |       - |
| Weekday Factor (non-learnable)   |          - |        - |    4.2924 |      V4 |
| Weekday Factor Model             |     1.7337 |   6.2968 |    1.2501 |      V5 |
| Fusion Fashion (concat)          |     4.4311 |   2.0399 |    1.4001 |      V9 |
| Fusion Fashion (add)             |     4.4655 |   2.0404 |    1.5238 |     V10 |
| Fusion Fashion (product)         |     4.4871 |   2.0026 |    2.6962 |     V11 |

In [ ]:
# Read data and aggregate into monthly data

import pandas as pd

train_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
# columns: date, date_block_num, shop_id, item_id, item_price, item_cnt_day
train_monthly_df = train_df.groupby(['date_block_num','shop_id','item_id']).agg(sale_cnt_month=('item_cnt_day','sum'),price_month=('item_price','mean')).reset_index()
train_monthly_df['shop_item'] = train_monthly_df['shop_id'].astype('string') + '-' + train_monthly_df['item_id'].astype('string')
shop_item_list = pd.unique(train_monthly_df['shop_item'])
shop_list = pd.unique(train_monthly_df['shop_id'])
item_list = pd.unique(train_monthly_df['item_id'])

print("count of shop_item:",len(shop_item_list))
print('------')
print('count of shop_id:', len(shop_list))
print('min shop_id:', min(shop_list))
print('max shop id:', max(shop_list))
print('------')
print('count item_id:', len(item_list))
print('min item_id:', min(item_list))
print('max item id:', max(item_list))
train_monthly_df

In [ ]:
# get the "sale_cnt sequences" of each "shop_item"
## get (data_seq_dict)

import numpy as np

data_seq_dict = {}
for shop_item in shop_item_list:
    data_seq_dict[shop_item] = [0.0 for i in range(0,34)]
for _, row in train_monthly_df.iterrows():
    data_seq_dict[row['shop_item']][int(row['date_block_num'])] = row['sale_cnt_month']
    
print(len(data_seq_dict))

In [ ]:
# get the "last price" of each "shop_item" as the future price
## shopitem_price_dict

import random

shopitem_price_dict = {}  # {shop_item: [avg_price, current_max_date_block_num, cnt]}
# record price and max date_block_num of each shop_item
for _, row in train_monthly_df.iterrows():
    if row['shop_item'] not in shopitem_price_dict.keys():
        shopitem_price_dict[row['shop_item']] = [row['price_month'], row['date_block_num'], 1]
    else:
        if row['date_block_num'] > shopitem_price_dict[row['shop_item']][1]:
            shopitem_price_dict[row['shop_item']] = [row['price_month'], row['date_block_num'], 1]
        if row['date_block_num'] == shopitem_price_dict[row['shop_item']][1]:
            entry = shopitem_price_dict[row['shop_item']]
            count = entry[2] + 1
            price = (entry[0] * entry[2] + row['price_month']) / count
            shopitem_price_dict[row['shop_item']] = [price, row['date_block_num'], count]
# remove date_block_num and count in the dict to only keep [shop_item, price_month]
for k,v in shopitem_price_dict.items():
    shopitem_price_dict[k] = v[0]

# UT code: printout a random sample to check
print(len(shopitem_price_dict))
idx = random.randint(0,len(shop_item_list))
shop_item = shop_item_list[idx]
print("The last price of shop-item", shop_item, ":", shopitem_price_dict[shop_item])
shop_item = shop_item.split('-')
train_monthly_df[(train_monthly_df['shop_id']==int(shop_item[0])) &
                 (train_monthly_df['item_id']==int(shop_item[1])) &
                 (train_monthly_df['price_month']>0.0)].sort_values(by=['date_block_num'],ascending=[False])

In [ ]:
# get the avg "sale_cnt sequences" of each "item" as default
# In case the shop_item in test dataset does not appear in training dataset, 
# we use the "item avg sale_cnt sequence" as default.
## get (item_seq_dict)

item_seq_dict = {}
item_seq_num_dict = {}
# make sum
for k, v in data_seq_dict.items():
    item = k.split('-')[1]
    if item not in item_seq_dict.keys():
        item_seq_dict[item] = v
        item_seq_num_dict[item] = 1
    else:
        item_seq_dict[item] = [i+j for i, j in zip(item_seq_dict[item], v)]
        item_seq_num_dict[item] += 1
# make average
for item in item_seq_dict.keys():
    item_seq_dict[item] = [element / item_seq_num_dict[item] for element in item_seq_dict[item]]

print(len(item_seq_dict))

In [ ]:
# get the "last price" of each "item" as the future price
## get (item_price_dict)

item_price_dict = {}  # {item_id: [avg_price, current_max_date_block_num, cnt]}
# record prices and max date_block_num of each item
for _, row in train_monthly_df.iterrows():
    item_id = str(row['item_id'])
    if item_id not in item_price_dict.keys():
        item_price_dict[item_id] = [row['price_month'], row['date_block_num'], 1]
    else:
        if row['date_block_num'] > item_price_dict[item_id][1]:
            item_price_dict[item_id] = [row['price_month'], row['date_block_num'], 1]
        if row['date_block_num'] == item_price_dict[item_id][1]:
            entry = item_price_dict[item_id]
            count = entry[2] + 1
            price = (entry[0] * entry[2] + row['price_month']) / count
            item_price_dict[item_id] = [price, row['date_block_num'], count]
# remove date_block_num and count in the dict to only keep [item_id, price_month]
for k,v in item_price_dict.items():
    item_price_dict[k] = v[0]
    
# UT code: printout a random sample to check
print(len(item_price_dict))
idx = random.randint(0,len(item_list))
item_id = str(item_list[idx])
print("The last price of item", item_id, ":", item_price_dict[item_id])
tmp_df = train_monthly_df[(train_monthly_df['item_id']==int(item_id))][['date_block_num', 'price_month']]
tmp_df = tmp_df[tmp_df['date_block_num']==tmp_df['date_block_num'].max()][['price_month']].mean()
print(tmp_df)

In [ ]:
# Data Preprocess

import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K


# parameter
ITEM_VOCAB_LEN = 1000

# Scaler of price
price_scaler = MinMaxScaler(feature_range=(0,1))
price_scaler.fit([[i] for i in train_monthly_df['price_month'].tolist()])

# Split X and Y
X = []
Y = []
for k,v in data_seq_dict.items():
    shop_item = k
    static_vec = np.array([int(i) for i in k.split('-')])
    static_vec[1] = static_vec[1] % ITEM_VOCAB_LEN
    price_vec = price_scaler.transform([[shopitem_price_dict[k]]]).reshape((-1,))
    static_vec = np.concatenate([static_vec, price_vec])
    seq_vec = np.array(v).reshape((-1,1))  # (steps, 1)
    X.append([shop_item, static_vec, seq_vec[:-1,:]])
    Y.append(seq_vec[-1,:])
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.3)
X_train_static = np.array([x[1] for x in X_train])
X_train_seq = np.array([x[2] for x in X_train])
Y_train = np.array(Y_train)
X_val_static = np.array([x[1] for x in X_val])
X_val_seq = np.array([x[2] for x in X_val])
Y_val = np.array(Y_val)

# Global Parameters
STEPS = X_train_seq.shape[1]

print("Training Dataset:")
print("\nX_train_static:")
print(X_train_static.shape)
print(X_train_static)
print("\nX_train_seq:")
print(X_train_seq.shape)
print(X_train_seq)
print("\nY_train:")
print(Y_train.shape)
print(Y_train)
print("\nValidation Dataset:")
print("\nX_val_static:")
print(X_val_static.shape)
print(X_val_static)
print("\nX_val_seq:")
print(X_val_seq.shape)
print(X_val_seq)
print("\nY_val:")
print(Y_val.shape)
print(Y_val)

In [ ]:
# # Initial Model, Wide&Deep

# # Model Parameters
# STEPS = X_train_seq.shape[1]

# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# from math import log
# import tensorflow as tf
# import tensorflow_probability as tfp
# from tensorflow import feature_column
# from tensorflow.nn import relu
# from tensorflow.keras import backend as K
# from tensorflow.keras.models import Sequential
# from tensorflow.keras import Model
# from tensorflow.keras import Input
# from tensorflow.keras.layers import Dense, Lambda, Concatenate, Reshape, Embedding, Conv1D, Add, Flatten, LSTM, Activation, BatchNormalization
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import mean_squared_error

# # Loss
# def root_mean_squared_error(y_true, y_pred):
#     return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# # Model
# # Wide Component
# input_static_vec = Input(name='input_static', shape = (3,))

# shop_vec = Lambda(lambda x: tf.slice(x, (0,0), (-1,1)))(input_static_vec)
# shop_emb_vec = Embedding(input_dim=60, output_dim=4, input_length=(1,))(shop_vec)
# shop_emb_vec = Flatten()(shop_emb_vec)
# item_vec = Lambda(lambda x: tf.slice(x, (0,1), (-1,1)))(input_static_vec)
# item_emb_vec = Embedding(input_dim=ITEM_VOCAB_LEN, output_dim=4, input_length=(1,))(item_vec)
# item_emb_vec = Flatten()(item_emb_vec)
# price_vec = Lambda(lambda x: tf.slice(x, (0,2), (-1,1)))(input_static_vec)
# wide_vec = Concatenate(axis=-1, name='Concat_wide')([shop_emb_vec, item_emb_vec, price_vec])

# # Deep Component
# input_seq_vec = Input(name='input_seq', shape = (STEPS,1))

# deep_vec = LSTM(units=4, name='LSTM')(input_seq_vec)

# # Fusion Layer
# output_vec = Concatenate(axis=-1, name='Concat_fusion')([wide_vec, deep_vec])
# output_vec = Dense(units=1, activation='relu')(output_vec)

# model = Model(inputs=[input_static_vec,input_seq_vec], outputs=[output_vec])
# model.compile(loss=root_mean_squared_error, optimizer=Adam(learning_rate=0.05))
# model.summary()

# callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001,
#                                               verbose=1, mode='min', patience=3,
#                                               restore_best_weights=True)]

# history = model.fit(x={'input_static': X_train_static, 'input_seq': X_train_seq}, 
#                     y=Y_train,
#                     validation_data=({'input_static':X_val_static,'input_seq':X_val_seq}, Y_val), 
#                     epochs=100, 
#                     batch_size=1024*1024,
#                     callbacks=callbacks)

In [ ]:
# # Weekday Factor Model

# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# from math import log
# import tensorflow as tf
# import tensorflow_probability as tfp
# from tensorflow import feature_column
# from tensorflow.nn import relu
# from tensorflow.keras import backend as K
# from tensorflow.keras.models import Sequential
# from tensorflow.keras import Model
# from tensorflow.keras import Input
# from tensorflow.keras.layers import Dense, Lambda, Concatenate, Reshape, Embedding, Conv1D, Add, Flatten, LSTM, Activation, BatchNormalization, Multiply
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import mean_squared_error

# # Loss
# def root_mean_squared_error(y_true, y_pred):
#     return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# # Model
# # Wide Component
# input_static_vec = Input(name='input_static', shape = (3,))

# shop_vec = Lambda(lambda x: tf.slice(x, (0,0), (-1,1)))(input_static_vec)
# shop_emb_vec = Embedding(input_dim=60, output_dim=4, input_length=(1,))(shop_vec)
# shop_emb_vec = Flatten()(shop_emb_vec)
# item_vec = Lambda(lambda x: tf.slice(x, (0,1), (-1,1)))(input_static_vec)
# item_emb_vec = Embedding(input_dim=ITEM_VOCAB_LEN, output_dim=4, input_length=(1,))(item_vec)
# item_emb_vec = Flatten()(item_emb_vec)
# price_vec = Lambda(lambda x: tf.slice(x, (0,2), (-1,1)))(input_static_vec)
# wide_vec = Concatenate(axis=-1, name='Concat_wide')([shop_emb_vec, item_emb_vec, price_vec])

# # Deep Component
# input_seq_vec = Input(name='input_seq', shape = (STEPS,1))

# deep_vec = LSTM(units=4, name='LSTM')(input_seq_vec)

# # Fusion Layer
# model_vec = Concatenate(axis=-1, name='Concat_fusion')([wide_vec, deep_vec])
# model_vec = Dense(units=1, activation='relu')(model_vec)

# # Weekday Factor Component
# # 1. calculate the Weekday Factor
# value_4week_vec = Lambda(lambda x: tf.slice(x, (0,STEPS-28,0), (-1,28,-1)), name='Weekday_Factor')(input_seq_vec)  # (N, 28, 1)
# value_4week_vec = Reshape((4,7))(value_4week_vec)  # (N, 4 ,7)
# week_mean_vec = Lambda(lambda x: tf.reduce_mean(x, axis=2, keepdims=True))(value_4week_vec)  # (N, 4, 1)
# day_factor_vec = Lambda(lambda x: tf.math.divide_no_nan(x[0],x[1]))([value_4week_vec,week_mean_vec])  # (N, 4, 7)
# weekday_factor_vec = Lambda(lambda x: tf.reduce_mean(x, axis=1))(day_factor_vec)  # (N, 7)
# # 2. calculate the Base (mean of last 3 weeks)
# value_3week_vec = Lambda(lambda x: tf.slice(x, (0,STEPS-21,0), (-1,21,-1)), name='base')(input_seq_vec)  # (N, 28, 1)
# value_3week_vec = Reshape((21,))(value_3week_vec)  # (N, 28)
# base_vec = Lambda(lambda x: tf.reduce_mean(x, axis=1, keepdims=True))(value_3week_vec)  # (N, 1)
# # 3. make predicrion (weekday_factor * base)
# weekday_factor_vec = Multiply()([weekday_factor_vec, base_vec])  # (N, 7)
# weekday_factor_vec = Lambda(lambda x: tf.slice(x, (0,0), (-1,1)))(weekday_factor_vec)  # (N, 1)

# # Output Layer
# output_vec = Concatenate(axis=-1, name='output_Concat')([model_vec, weekday_factor_vec])
# output_vec = Dense(units=1, activation='relu')(output_vec)

# model = Model(inputs=[input_static_vec,input_seq_vec], outputs=[output_vec])
# model.compile(loss=root_mean_squared_error, optimizer=Adam(learning_rate=0.05))
# model.summary()

# callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001,
#                                               verbose=1, mode='min', patience=3,
#                                               restore_best_weights=True)]

# history = model.fit(x={'input_static': X_train_static, 'input_seq': X_train_seq}, 
#                     y=Y_train,
#                     validation_data=({'input_static':X_val_static,'input_seq':X_val_seq}, Y_val), 
#                     epochs=100, 
#                     batch_size=1024*1024,
#                     callbacks=callbacks)

In [ ]:
# # Fusion Fashion Model (concat/add/product)

# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# from math import log
# import tensorflow as tf
# import tensorflow_probability as tfp
# from tensorflow import feature_column
# from tensorflow.nn import relu
# from tensorflow.keras import backend as K
# from tensorflow.keras.models import Sequential
# from tensorflow.keras import Model
# from tensorflow.keras import Input
# from tensorflow.keras.layers import Dense, Lambda, Concatenate, Reshape, Embedding, Conv1D, Add, Flatten, LSTM, Activation, BatchNormalization, Multiply
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import mean_squared_error

# # Loss
# def root_mean_squared_error(y_true, y_pred):
#     return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# # Model
# # Wide Component
# input_static_vec = Input(name='input_static', shape = (3,))

# shop_vec = Lambda(lambda x: tf.slice(x, (0,0), (-1,1)))(input_static_vec)
# shop_emb_vec = Embedding(input_dim=60, output_dim=4, input_length=(1,))(shop_vec)
# shop_emb_vec = Flatten()(shop_emb_vec)
# item_vec = Lambda(lambda x: tf.slice(x, (0,1), (-1,1)))(input_static_vec)
# item_emb_vec = Embedding(input_dim=ITEM_VOCAB_LEN, output_dim=4, input_length=(1,))(item_vec)
# item_emb_vec = Flatten()(item_emb_vec)
# price_vec = Lambda(lambda x: tf.slice(x, (0,2), (-1,1)))(input_static_vec)
# wide_vec = Concatenate(axis=-1, name='Concat_wide')([shop_emb_vec, item_emb_vec, price_vec])  # (N, 9)

# # Deep Component
# input_seq_vec = Input(name='input_seq', shape = (STEPS,1))

# deep_vec = LSTM(units=9, name='LSTM')(input_seq_vec)  # (N, 9)

# # Fusion Layer
# # output_vec = Concatenate(axis=-1, name='Concat_fusion')([wide_vec, deep_vec])
# # output_vec = Add(name='Concat_fusion')([wide_vec, deep_vec])
# # output_vec = Multiply(name='Concat_fusion')([wide_vec, deep_vec])
# output_vec = Dense(units=1, activation='relu')(output_vec)

# model = Model(inputs=[input_static_vec,input_seq_vec], outputs=[output_vec])
# model.compile(loss=root_mean_squared_error, optimizer=Adam(learning_rate=0.05))
# model.summary()

# callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001,
#                                               verbose=1, mode='min', patience=3,
#                                               restore_best_weights=True)]

# history = model.fit(x={'input_static': X_train_static, 'input_seq': X_train_seq}, 
#                     y=Y_train,
#                     validation_data=({'input_static':X_val_static,'input_seq':X_val_seq}, Y_val), 
#                     epochs=100, 
#                     batch_size=1024*1024,
#                     callbacks=callbacks)

In [ ]:
tf.keras.utils.plot_model(model, to_file='model_graph.png', show_shapes=True, show_layer_names=True)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.show()

In [ ]:
plt.plot(history.history['val_loss'])
plt.show()

In [ ]:
# Prepare Test dataset

test_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv').set_index('ID')
test_df['shop_item'] = test_df['shop_id'].astype('string') + '-' + test_df['item_id'].astype('string')

X_test_static = []
X_test_seq = []
for _, row in test_df.iterrows():
    shop_item = row['shop_item']
    shop_id = str(row['shop_id'])
    item_id = str(row['item_id'])
    
    # get sale_cnt_seq
    if shop_item in data_seq_dict.keys():
        sale_cnt_seq = data_seq_dict[shop_item][1:]
    else:
        if item_id in item_seq_dict.keys():
            sale_cnt_seq = item_seq_dict[item_id][1:]
        else:
            sale_cnt_seq = [0.0 for i in range(0,STEPS)]
    
    # get price
    if shop_item in shopitem_price_dict.keys():
        price = shopitem_price_dict[shop_item]
    else:
        if item_id in item_price_dict.keys():
            price = item_price_dict[item_id]
        else:
            price = 0.0
    
    # generate the sample
    static_vec = np.array([int(shop_id), int(item_id) % ITEM_VOCAB_LEN])
    price_vec = price_scaler.transform([[price]]).reshape((-1,))
    static_vec = np.concatenate([static_vec, price_vec])
    seq_vec = np.array(sale_cnt_seq).reshape((-1,1))  # (steps, 1)
    X_test_static.append(static_vec)
    X_test_seq.append(seq_vec)
X_test_static = np.array(X_test_static)
X_test_seq = np.array(X_test_seq)


print('X_test_static:')
print(X_test_static.shape)
print(X_test_static)
print('\nX_test_seq:')
print(X_test_seq.shape)
print(X_test_seq)

In [ ]:
pred = model.predict({'input_static':X_test_static, 'input_seq':X_test_seq})
print(pred.shape)
print(pred)

In [ ]:
# # Weekday Factor Method (Best Solver Poster)
# # non-Learnable model

# from scipy.stats import norm

# data = X_test_seq.squeeze()
# # Preprocess: recover outliers
# for i in range(0,data.shape[0]):
#     for j in range(13,data.shape[1]):
#         mean, stddev = norm.fit(data[i,j-13:j+1])
#         if data[i,j] > mean + 2 * stddev:
#             data[i,j] = mean + 2 * stddev
#         if data[i,j] < mean - 2 * stddev:
#             data[i,j] = mean - 2 * stddev

# # 1. calculate the Weekday Factor
# data_last_4_week = data[:,5:]  # (214200, 28)
# day_factor_list = []
# for i in range(0,4):
#     day_value = data_last_4_week[:,i*7:(i+1)*7]  # (214200, 7)
#     week_mean = np.mean(day_value, axis=1, keepdims=True)  # (214200, 1)
#     day_factor = np.divide(day_value, week_mean, out=np.zeros_like(day_value), where=week_mean!=0.)
#     day_factor_list.append(day_factor)
# day_factors = np.stack(day_factor_list, axis=2)  # (214200, 7, 4)
# weekday_factor = np.mean(day_factors, axis=2)  # (214200, 7)

# # 2. calculate the Base (mean of last 3 weeks)
# data_last_3_week = data[:,12:]  # (214200, 21)
# base = np.mean(data_last_3_week, axis=1, keepdims=True)  # (214200, 1)

# # 3. make predicrion (weekday_factor * base)
# pred = weekday_factor * base  # (214200, 7)

In [ ]:
# recover the prediction (de-normalized)
results = pred[:,0]
results[results<0.] = 0.
results[0:10]

In [ ]:
submission_df = pd.DataFrame(results, columns=['item_cnt_month'])
submission_df.index.name = 'ID'
submission_df

In [ ]:
submission_df.to_csv('/kaggle/working/submission.csv')